In [14]:
import os
import dotenv
from pathlib import Path

from langchain_cohere import CohereEmbeddings
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import (
    WebBaseLoader, 
    PyPDFLoader, 
    Docx2txtLoader,
)
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_cohere import ChatCohere
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

dotenv.load_dotenv()

True

In [3]:
# Load docs

doc_paths = [
    "docs/test_rag.pdf",
    "docs/test_rag.docx",
]

docs = [] 
for doc_file in doc_paths:
    file_path = Path(doc_file)

    try:
        if doc_file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif doc_file.endswith(".docx"):
            loader = Docx2txtLoader(file_path)
        elif doc_file.endswith(".txt") or doc_file.name.endswith(".md"):
            loader = TextLoader(file_path)
        else:
            print(f"Document type {doc_file.type} not supported.")
            continue

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading document {doc_file.name}: {e}")


# Load URLs

url = "https://docs.streamlit.io/develop/quick-reference/release-notes"
try:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())

except Exception as e:
    print(f"Error loading document from {url}: {e}")

In [4]:
docs

[Document(metadata={'source': 'docs/test_rag.pdf', 'page': 0}, page_content='My favorite food is margarita pizza.  \nThere are 47588 bottles in the tr uck.  '),
 Document(metadata={'source': 'docs/test_rag.docx'}, page_content='My favorite food is margarita pizza.\n\nThere are 47588 bottles in the truck.'),
 Document(metadata={'source': 'https://docs.streamlit.io/develop/quick-reference/release-notes', 'title': 'Release notes - Streamlit Docs', 'description': 'A changelog of highlights and fixes for each version of Streamlit.', 'language': 'No language found.'}, page_content="Release notes - Streamlit DocsDocumentationsearchSearchrocket_launchGet startedInstallationaddFundamentalsaddFirst stepsaddcodeDevelopConceptsaddAPI referenceaddTutorialsaddQuick referenceremoveCheat sheetRelease notesremove2025202420232022202120202019Pre-release featuresRoadmapopen_in_newweb_assetDeployConceptsaddStreamlit Community CloudaddSnowflakeOther platformsaddschoolKnowledge baseFAQInstalling dependencies

In [5]:
# Split docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=1000,
)

document_chunks = text_splitter.split_documents(docs)

In [8]:
# Tokenize and load the documents to the vector store
embeddings = CohereEmbeddings(
    model="embed-multilingual-v3.0",
)
vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding=embeddings,
)

In [9]:
# Retrieve

def _get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get inforamtion relevant to the conversation, focusing on the most recent messages."),
    ])
    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

    return retriever_chain

In [12]:
def get_conversational_rag_chain(llm):
    retriever_chain = _get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """You are a helpful assistant. You will have to answer to user's queries.
        You will have some context to help with your answers, but now always would be completely related or helpful.
        You can also use your knowledge to assist answering the user's queries.\n
        {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [17]:
# Augmented Generation
# llm_stream_openai = ChatOpenAI(
#     model="gpt-4o",  # Here you could use "o1-preview" or "o1-mini" if you already have access to them
#     temperature=0.3,
#     streaming=True,
# )

# llm_stream_anthropic = ChatAnthropic(
#     model="claude-3-5-sonnet-20240620",
#     temperature=0.3,
#     streaming=True,
# )

import time

llm_stream_cohere = ChatCohere(
    model="command-r7b-12-2024",
    temperature=0.3,
    streaming=True,
)

llm_stream = llm_stream_cohere  # Select between OpenAI and Anthropic models for the response

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "What is the latest version of Streamlit?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)
    time.sleep(0.05)
messages.append({"role": "assistant", "content": response_message})

The latest version of Streamlit as of February 4, 2025, is 1.42.0. This version includes several highlights, notable changes, and bug fixes. If you're looking to upgrade, you can do so using the following command:

```bash
pip install --upgrade streamlit
```

Let me know if you'd like to know more about any specific features or changes in this version!

In [ ]:
# import os
# from langchain_openai import AzureChatOpenAI
# import dotenv

# dotenv.load_dotenv()

# llm_stream = AzureChatOpenAI(
#     azure_endpoint=os.getenv("AZ_OPENAI_ENDPOINT"),
#     openai_api_version="2024-02-15-preview",
#     model_name="gpt-4o",
#     openai_api_key=os.getenv("AZ_OPENAI_API_KEY"),
#     openai_api_type="azure",
#     temperature=0.3,
#     streaming=True,
# )

# prompt = "Tell me something about Azure"

# for chunk in llm_stream.stream(prompt):
#     print(chunk.content, end="", flush=True)

Sure! Microsoft Azure, commonly referred to as Azure, is a cloud computing platform and service created by Microsoft. It provides a wide range of cloud services, including those for computing, analytics, storage, and networking. Users can choose and configure these services to develop and scale new applications or run existing applications in the public cloud.

Here are some key aspects of Azure:

1. **Compute Services**: Azure offers virtual machines, containers, and serverless computing options. This includes Azure Virtual Machines, Azure Kubernetes Service (AKS), and Azure Functions.

2. **Storage Solutions**: Azure provides scalable cloud storage solutions such as Azure Blob Storage, Azure Disk Storage, and Azure File Storage. These services cater to different data storage needs, from unstructured data to persistent disk storage.

3. **Networking**: Azure offers a variety of networking services, including Azure Virtual Network, Azure Load Balancer, and Azure Application Gateway. Th